# Lab 6 - Databases and Large Data

So far in the course we have looked at various techniques for dealing with data, manipulating text and files and even how to scrape websites for data. But sometimes we want data from a pre-existing dataset such as those provided by [Statistics Canada](https://www.statcan.gc.ca/eng/start) or other data collecting agencies. The way we access these datasets is by downloading them from their respective websites or we can sometimes use an API, or application program interface, to import the data directly into our code.

Other ways of getting data is from a database. Databases are ways to store lots of data and they come with a query language. A query language allows users to pull data from the database using various keywords.

In this lab we will look at how to use APIs and how to turn them into a database so that we can practise using a query language.

The data set that we will be working with today is the [US 2013 language survey](https://www.census.gov/data/developers/data-sets/language-stats.html). We will learn how to call the API and get the language statistics for a specific language. Once we have this data we will turn it into a mySQL database.

The hardest part about obtaining data from an API call is structuring the API call correctly. For the API that we will work with the base link that we will use is `https://api.census.gov/data/2013/language?` This is known as the API call and we will be adding our arguments after the question mark. In order to get data we need to specify what we want, in our case we want to get the number of speakers, the language and the state name. To get this we add `get=EST,LANLABEL,NAME`. To find out what each label does we could reference the [API documentation](https://api.census.gov/data/2013/language/variables.html). Finally we need to specify for which state we want the data and what language we want. We add `&for=state:*` where the * means we want all states and we add `&LAN=625` to get the Spanish language data. You can reference the [language codes document](https://www2.census.gov/programs-surveys/demo/about/language-use/primary_language_list.pdf) to find the correct code for each language. 

In the following code block we put all this information together to get our data.

In [ ]:
# we import the requests library as we will use this to obtain the data from the API
import requests

# we get a response from the api using requests.get() and store it in the response variable
response = requests.get("https://api.census.gov/data/2013/language?get=EST,LANLABEL,NAME&for=state:*&LAN=625")

# we can check if everything went correctly by printing the status code. 200 means everything went well
print("API status code:", response.status_code)

# we can also print the raw contents of our requests by calling content
# comment the following line out if you want to see the contents
print(response.content)

The data that we get from the the API call is very structured and easy to work with. However if we just call the content there are some artefacts that we dont want such as the "\n" in between all the lists. To get rid of this we will use something called JSON. You can think of JSON files as normal text files that are structured in a specific way that is easy to work with. To read up on JSON files you can follow the tutorial [here](https://beginnersbook.com/2015/04/json-tutorial/). While it is written from the perspective of a Javascript developer, it is still a good introduction, so don't worry about some of the particular code snippets you may not fully understand.

In the following code snippet we will turn the raw data that we receive from the API into a workable Python list.

In [ ]:
# import the JSON module
import json

# save the JSON data from the API into a python list
data = response.json()

# each item in the list, except the first, is a state
# we can iterate over each item and print it
for state in data:
    print(state)

As you see above, getting data from an API is fairly simple and returns the data in a very structured format. Finish the following code block in which I have provided some skeleton code and retrieve French language statistics from the API.

In [ ]:
# request the french data, hint: change the url used above to get the correct data
# you'll need to look up the French code in the document above

french_request = requests.get(" ") # insert the right URL

# check if the response is correct
print("API status code:", response.status_code)

# format the data using JSON
frenchData = #COMMAND HERE

for state in frenchData:
    print(state)

We could stop here and start analyzing our data since it is in a very workable fromat. For this lab, however, the focus isn't on analyzing data but it is about how to manage and store big ammounts of data. For this we will start to learn how to use a database. Databases are used to store data in a structured and easy to use format. A database is kind of like a csv in that it is a big table with rows and columns. In this lab we will use SQLite to introduce ourselves to database setup and handling.

To use the SQLite database that we will be using you need to know some of the basic commands and syntax of SQL. Read through the following tutorial to learn more about SQL and its commands: [SQL guide](https://learntocodewith.me/posts/sql-guide/). Now that you know what SQL is we can get into creating a database and creating our first table to store the language statistics we pulled from the API. 

One difference with SQLite is how we create a database. We dont need to call the `CREATE DATABASE` statement like it says in the SQL guide you just read. Instead of calling a statement we just connect to a database which is created automatically if it doesn't exist yet.

In [ ]:
# import SQLite
import sqlite3

# our first step is to connect to our database file
# this step replaces the "CREATE DATABASE" step
conn = sqlite3.connect("./test.db")

# this is our SQL statement that we will call through python
tableStatement = """CREATE TABLE IF NOT EXISTS spanishData (
                        numSpeakers integer, 
                        language text, 
                        state text, 
                        languageCode integer,
                        stateCode integer PRIMARY KEY);"""

# the cursor 'c' allows us to execute commands on the database
c = conn.cursor()
c.execute(tableStatement)

We have created a table in the last code block, so let's explore the tableStatement that we used to create it. "CREATE TABLE" is the statement that we call and everything else are the arguments. "IF NOT EXISTS" makes sure we do not create a duplicate table called "spanishData". Inside of the parentheses are the table headers, so we created columns for item from our API data. Notice the "PRIMARY KEY" label after the stateCode. This label lets the table know that this item in the list must be unique for each data point. Because this item will be unique for each data point, we can use it in the future if we need to change data for a specific data point. Now let's look at how to add our data to the database.

Statements like the one we used to create the table are very important for database management. These SQL statements are the only way we can interact with the database. Python just helps us execute these SQL statements. 

In [ ]:
# the SQL INSERT statement that we will use to add data to the database
sql = """INSERT INTO spanishData(
            numSpeakers, language, state, languageCode, stateCode)
            VALUES(?,?,?,?,?)"""

# loop over each data point from the API
# notice we skip data[0] since this contains the original headers from the API
for i in range(1, 53):
    c.execute(sql, data[i])
    
# finally we commit to the database
c.commit()

Now let's look at how we can update a data point and then subsequently delete it. Let's pretend a new survey was run in Puerto Rico (state code=72) and found out that there are actually 4,100,000 Spanish speakers. We will update our data entry for Puerto Rico with that number first and then we will delete the entry.

In [ ]:
# first we write our SQL statement
sql = """UPDATE spanishData 
         SET numSpeakers = 4100000
         WHERE stateCode = 72"""

# notice we only pass one argument to the execute command, namely our SQL statement
# this is because we entered all the data directly into the command instead of 
# using the placeholder '?'
c.execute(sql)
conn.commit()

# You can also entirely delete the Puerto Rico entry or any other entry
# Do one or the other (above to update, below to delete)
# delSQL = """DELETE FROM spanishData
#            WHERE stateCode = 72"""
            
# c.execute(delSQL)
# conn.commit()

You might be wondering how we can see if any of these commands actually worked, well, this is where we can use the `SELECT` statement. This statement allows you to select one or multiple data points from your database. Let's look at how to retrieve one data point and all of the data from the database.

In [ ]:
# first we select all data points from spanishData
# * here will select all columns for each row
selectAll = "SELECT * FROM spanishData"

# execute the SQL statement
c.execute(selectAll)

# we retrieve the data we selected from the cursor
data = c.fetchall()

# now we can loop through the data and print it
for row in data:
    print(row)

In [ ]:
# to fetch only one item we just add the WHERE statement
# we can either replace the ? by the code for the state we want
# or we can pass the state we want as an argument to execute()
selectOne = "SELECT * FROM spanishData WHERE stateCode=?"

c.execute(selectOne, (2,))
data = c.fetchall()

print(data)

# at the end, close the db
# this close command is commented, because you will need it open for the final task below

# c.close()

For learning more about SQL and the various statements it supports you can read through the tutorial from w3schools: [SQL](https://www.w3schools.com/sql/). Just keep in mind that we are using SQLite, which doesn't necessarily support some of the more advanced statements such as the ones listed [here](https://www.sqlite.org/omitted.html) but don't worry about those since you will most likely not use them anytime soon.

For the final task you will create a new table in the languageData database for the French data that you retrieved. I have set up some skeleton code for you to stat working with.

In [ ]:
# since you are still connected to the database you wont need to create a new connection
# (except if you closed it above)

# start with writing the SQL statement for creating a new table
frenchTable = """ """

# the cursor should also still be active so we can just call the execute function
c.execute()

# write the insert statement for the frenchData
insert = """ """

# fill in the correct range and the code inside the for loop
# remember we want to skip frenchData[0] since those are old headers
for i in range():
    
    
# after the commands are ran we need to commit to the database
c.commit()

In [ ]:
# close the db

c.close()